<a href="https://colab.research.google.com/github/syedmahmoodiagents/RAG/blob/main/NewRAG_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [24]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface import ChatHuggingFace

In [28]:
import os, getpass

In [29]:
os.environ['HF_TOKEN'] = getpass.getpass('HuggingFace Token:')

HuggingFace Token:··········


In [30]:
from langchain_community.vectorstores import FAISS

In [31]:
from langchain_core.documents import Document

In [49]:
plain_texts = [
    "LangChain helps build RAG applications.",
    "RAG stands for Retrieval Augmented Generation.",
    "FAISS is used for vector similarity search."
]

documents = [Document(page_content=text) for text in plain_texts]

In [50]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [51]:
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

In [52]:
llm=ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b"))

In [53]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [54]:
# Define a prompt template
template = """Given the following context and a question, generate a concise answer based solely on the provided context. If the answer cannot be found in the context, respond with "I don't know."

Context:
{context}

Question: {question}

Concise Answer:
"""


In [55]:
prompt = ChatPromptTemplate.from_template(template)

In [56]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [57]:
retrv_chain = retriever | format_docs

In [58]:
rag_chain = (
    {"context": retrv_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [59]:
query = "What does RAG stand for?"
result = rag_chain.invoke(query)

print("Answer:\n", result)

Answer:
 Retrieval Augmented Generation.
